# 安装

安装`pytorch`作为神经网络框架和`pretty_mid`用于处理`midi`文件。
以及`numpy`等其他常用`python`库。

数据集使用`Tensorflow`[官方教程](https://tensorflow.google.cn/tutorials/audio/music_generation)中的 [`maestro-v2.0.0`]( https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip )

In [2]:
import glob
import torch
import torch.utils.data as tud
import pretty_midi as pm
import numpy as np
from tqdm import  tqdm
import random
from typing import *

# 读取Midi文件

读取midi文件

In [2]:
midifile = "/home/sid/Downloads/m_basho.mid"
pmf = pm.PrettyMIDI(midifile)

`instruments` 为乐器的列表

`instrument` 等于`pmf`的第一个乐器，类型为`pretty_midi.Instrument`, `instrument.notes` 即为该乐器的音符列表。

In [4]:
instrument = pmf.instruments[0]
instrument.notes

[Note(start=1.000000, end=1.500000, pitch=87, velocity=60),
 Note(start=1.500000, end=2.000000, pitch=85, velocity=60),
 Note(start=2.000000, end=2.500000, pitch=87, velocity=60),
 Note(start=2.500000, end=3.000000, pitch=89, velocity=60),
 Note(start=3.000000, end=4.000000, pitch=90, velocity=60),
 Note(start=4.000000, end=4.500000, pitch=87, velocity=60),
 Note(start=4.500000, end=5.000000, pitch=90, velocity=60),
 Note(start=5.000000, end=5.500000, pitch=89, velocity=60),
 Note(start=5.500000, end=6.000000, pitch=85, velocity=60),
 Note(start=6.000000, end=6.500000, pitch=85, velocity=60),
 Note(start=6.500000, end=7.000000, pitch=82, velocity=60),
 Note(start=7.000000, end=9.000000, pitch=85, velocity=60),
 Note(start=10.000000, end=11.000000, pitch=83, velocity=60),
 Note(start=11.000000, end=12.000000, pitch=85, velocity=60),
 Note(start=12.000000, end=13.000000, pitch=83, velocity=60),
 Note(start=13.000000, end=13.500000, pitch=82, velocity=60),
 Note(start=13.500000, end=14.00

`pretty_midi.Note`有4个属性

    - start    : 开始时间
    - end      : 结束时间
    - pitch    : 音高
    - velocity : 音符力度


创建midi文件使用`pmf.write(midi_file)`即可。

# 处理Midi数据

从 `instrument` 中读取到的音符列表中音符的三个特征`pitch`、`step`、`duration`将会被处理转换为`numpy`数组。

In [3]:
def GetNoteSequence(instrument: pm.Instrument) -> np.ndarray:
    """
    先根据音符的开始时间排序;
    由音符的开始时间减去前一个音符的开始时间得到`step`;
    由音符的结束时间减去开始时间得到`duration`；
    """
    sorted_notes = sorted(instrument.notes, key=lambda x: x.start)
    assert len(sorted_notes) > 0

    prev_starts = [note.start for note in sorted_notes]
    prev_starts = [prev_starts[0]] + prev_starts[:-1]

    notes = [
        [note.pitch, note.start - prev_start, note.end - note.start]
        for note, prev_start in zip(sorted_notes, prev_starts)
    ]

    return np.array(notes)

In [16]:
GetNoteSequence(instrument)

array([[87. ,  0. ,  0.5],
       [85. ,  0.5,  0.5],
       [87. ,  0.5,  0.5],
       [89. ,  0.5,  0.5],
       [90. ,  0.5,  1. ],
       [87. ,  1. ,  0.5],
       [90. ,  0.5,  0.5],
       [89. ,  0.5,  0.5],
       [85. ,  0.5,  0.5],
       [85. ,  0.5,  0.5],
       [82. ,  0.5,  0.5],
       [85. ,  0.5,  2. ],
       [83. ,  3. ,  1. ],
       [85. ,  1. ,  1. ],
       [83. ,  1. ,  1. ],
       [82. ,  1. ,  0.5],
       [80. ,  0.5,  0.5],
       [82. ,  0.5,  0.5],
       [83. ,  0.5,  0.5],
       [85. ,  0.5,  1. ],
       [82. ,  1. ,  1. ],
       [87. ,  1. ,  0.5],
       [85. ,  0.5,  0.5],
       [87. ,  0.5,  0.5],
       [89. ,  0.5,  0.5],
       [90. ,  0.5,  1. ],
       [87. ,  1. ,  0.5],
       [90. ,  0.5,  0.5],
       [89. ,  0.5,  0.5],
       [85. ,  0.5,  0.5],
       [85. ,  0.5,  0.5],
       [82. ,  0.5,  0.5],
       [85. ,  0.5,  2. ],
       [83. ,  3. ,  1. ],
       [87. ,  0. ,  1. ],
       [89. ,  1. ,  1. ],
       [85. ,  0. ,  1. ],
 

# 准备训练数据

pyTorch中数据处理的核心是`torch.utils.data.Dataset`和`torch.utils.data.DataLoader`
我们需要自定义一个Dataset来处理我们的midi数据
首先继承`torch.utils.data.Dataset`，然后实现`__init__`,`__getitem__`,`__len__`三个函数，功能分别为初始化，取出第i个数据，获得数据总数


在初始化中，首先用`glob`读出文件列表（`glob`可以使用通配符），然后遍历所有文件，用`pretty_midi`打开，得到它的音符序列，并对音符的`pitch`归一化（`tensorflow`官方教程这样干的）
将音符序列保存在类里面

（关于`np.append`的功能可以看这个[文章](https://blog.csdn.net/sinat_28576553/article/details/90247286)）


在读取第`i`个数据时，取出第`i`个音符开始的，长度为`seq_len`的序列作为输入数据，取出序列尾部的下一个音符为标签


最后加了`getendseq`功能，主要是方便测试时能够获得无标签的最后一个序列
<!-- ————————————————
版权声明：本文为CSDN博主「CaptainHarryChen」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/can919/article/details/122793127 -->

In [4]:
class SequenceMIDI(tud.Dataset):
    def __init__(self, files, seq_len, max_file_num=None):
        notes = None

        filenames = glob.glob(files)
        print(f"Find {len(filenames)} files.")

        if max_file_num is None:
            max_file_num = len(filenames)
        print(f"Reading {max_file_num} files...")

        for f in tqdm(filenames[:max_file_num]): # tqdm 提供进度条        
            pmf = pm.PrettyMIDI(f)
            instrument = pmf.instruments[0]
            new_notes = GetNoteSequence(instrument)
            new_notes /= [128.0, 1.0, 1.0]
            if notes is not None:
                notes = np.append(notes, new_notes, axis=0)
            else:
                notes = new_notes

        self.seq_len = seq_len
        self.notes = np.array(notes, dtype=np.float32)
    
    def __len__(self):
        return len(self.notes) - self.seq_len

    def __getitem__(self, idx) -> Tuple[np.ndarray, dict]:
        label_note = self.notes[idx + self.seq_len]
        label = {
            'pitch': (label_note[0] * 128).astype(np.int64), 
            'step': label_note[1],
            'duration': label_note[2]
        }
        return self.notes[idx:idx+self.seq_len], label
    
    def getendseq(self) -> np.ndarray:
        return self.notes[-self.seq_len:]

# 加载数据

用刚刚构建的Dataset来构建一个DataLoader作为数据加载器
然后就可以像遍历数组一样遍历DataLoader来获取数据了

In [5]:
# 一些资源
batch_size = 64
sequence_lenth = 25
max_file_num = 200
epochs = 200
learning_rate = 0.005

loss_weight = [0.1, 20.0, 1.0]

save_model_name = "music_producter.pth"

# 使用 GPU 训练如果可以
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device.")


Using cpu device.


In [6]:
trainning_data = SequenceMIDI(
    "maestro-v2.0.0/*/*.midi", 
    sequence_lenth,
    max_file_num=max_file_num
)
print(f"Read {len(trainning_data)} sequences.")
loader = tud.DataLoader(trainning_data, batch_size=batch_size)

for X, y in loader:
    print(f"X: {X.shape} {X.dtype}")
    print(f"y: {y}")
    break

Find 1282 files.
Reading 200 files...


100%|██████████| 200/200 [01:34<00:00,  2.12it/s]

Read 1166215 sequences.
X: torch.Size([64, 25, 3]) torch.float32
y: {'pitch': tensor([72, 63, 56, 73, 72, 65, 61, 70, 72, 65, 61, 70, 65, 61, 72, 70, 60, 63,
        68, 70, 60, 68, 63, 67, 68, 60, 63, 70, 68, 63, 60, 80, 84, 82, 80, 79,
        77, 75, 73, 72, 70, 68, 67, 65, 56, 49, 56, 49, 49, 58, 67, 67, 48, 51,
        48, 51, 68, 72, 70, 53, 49, 68, 67, 65]), 'step': tensor([9.3750e-03, 3.8438e-01, 5.2083e-03, 1.6250e-01, 1.8438e-01, 3.6667e-01,
        6.2500e-03, 2.2917e-01, 7.0833e-02, 5.7292e-02, 1.0417e-03, 1.0417e-03,
        3.8958e-01, 1.6667e-02, 1.7604e-01, 2.0521e-01, 3.7292e-01, 1.5625e-02,
        2.0312e-01, 9.3750e-02, 5.0000e-02, 1.0417e-03, 6.2500e-03, 2.7500e-01,
        8.0208e-02, 9.3750e-03, 5.2083e-03, 3.1563e-01, 1.1146e-01, 2.2917e-02,
        2.0833e-03, 1.4854e+00, 1.1458e-01, 9.6875e-02, 1.0833e-01, 8.6458e-02,
        1.0938e-01, 1.1042e-01, 9.3750e-02, 1.0625e-01, 1.0625e-01, 9.2708e-02,
        1.1667e-01, 1.6042e-01, 3.8750e-01, 5.2083e-03, 3.6875e-

# 模型构建

继承`torch.nn.Module`来构建自己的模型


一个`LSTM`处理输入的音符，再分别用三个全连接层算出`pitch`,`step`,`duration`，其中`pitch`的全连接层后使用`Sigmoid`将得到的值放在0~1之间


在这里`LSTM`在指定`batch_first=True`时，输入维度为 $(N,L,H_{in})$，分别为`batch`，序列长度，输入维度

`pitch`特征输出为128位，表示每个音高出现的权重
`step`和`duration`都是一维的标量

一般`torch.nn.`中的自带模型初始化都是`(输入维度，输出维度)`

在`forward`中，注意到`LSTM`的输出为两个，它的输出格式其实是 $output,(h_n,c_n)$，后面的**tuple**是我们用不到的隐藏状态
而`output`是一个序列，而我们只需要序列的最后一个，所以后面的`x`都是取`x[:-1]`
（具体可以去了解LSTM的原理）
<!-- ————————————————
版权声明：本文为CSDN博主「CaptainHarryChen」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/can919/article/details/122793127 -->

In [13]:
class MusicProducter(torch.nn.Module):
    def __init__(self):
        super(MusicProducter, self).__init__()
        self.lstm = torch.nn.LSTM(3, 128, num_layers=1, batch_first=True)
        self.pitch_linear = torch.nn.Linear(128, 128)
        self.pitch_sigmoid = torch.nn.Sigmoid()
        self.step_linear = torch.nn.Linear(128, 1)
        self.duration_linear = torch.nn.Linear(128, 1)
    
    def forward(self, x, hidden_prev):
        x, hidden_prev = self.lstm(x, hidden_prev)
        pitch = self.pitch_sigmoid(self.pitch_linear(x[:, -1]))
        # print(f"row pitch predict data: {pitch}")
        step = self.step_linear(x[:, -1])
        duration = self.duration_linear(x[:, -1])
        return {'pitch': pitch, 'step': step, 'duration': duration}, hidden_prev

In [8]:
class MusicProducterExport(torch.nn.Module):
    def __init__(self):
        super(MusicProducterExport, self).__init__()
        self.lstm = torch.nn.LSTM(3, 128, num_layers=1, batch_first=True)
        self.pitch_linear = torch.nn.Linear(128, 128)
        self.pitch_sigmoid = torch.nn.Sigmoid()
        self.step_linear = torch.nn.Linear(128, 1)
        self.duration_linear = torch.nn.Linear(128, 1)
    
    def forward(self, x, hidden_prev):
        x, hidden_prev = self.lstm(x)
        pitch = self.pitch_sigmoid(self.pitch_linear(x[:, -1]))
        step = self.step_linear(x[:, -1])
        duration = self.duration_linear(x[:, -1])
        return (pitch, step, duration), hidden_prev

# 损失函数

为了方便我把损失函数也写成一个模型
在这个损失函数中，分别计算三个特征的损失，并带权加和
`pitch`使用交叉熵（常用于分类器），而另外两个标量用均方差并带上使他变为正数的压力（毕竟时间都是正数）

注意到`torch`自带的`CrossEntropyLoss`既可以计算一个分类权重与下标的交叉熵，也可以两个分类权重的交叉熵
也就是下面两种都支持计算（这里我们使用第一种）
<!-- ————————————————
版权声明：本文为CSDN博主「CaptainHarryChen」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/can919/article/details/122793127 -->

```
[        1      ,        2       ,        3       ]
[[0.05, 0.95, 0], [0.1, 0.8, 0.1], [0.3, 0.7, 0.0]]
```


```
[[1.00, 0.00, 0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
[[0.05, 0.95, 0], [0.1, 0.8, 0.1], [0.3, 0.7, 0.0]]
```

In [9]:
def mse_with_positive_pressure(pred, y):
    mse = (y-pred) ** 2
    positive_pressure = 10 * torch.maximum(-pred, torch.tensor(0))
    return torch.mean(mse + positive_pressure)

In [10]:
class MPLoss(torch.nn.Module):
    def __init__(self, weight):
        super(MPLoss, self).__init__()
        self.weight = torch.Tensor(weight)
        self.pitch_loss = torch.nn.CrossEntropyLoss()
        self.step_loss = mse_with_positive_pressure
        self.duration_loss = mse_with_positive_pressure
    
    def forward(self, pred, y):
        losses = [
            self.pitch_loss(pred['pitch'], y['pitch']),
            self.step_loss(pred['step'], y['step']),
            self.duration_loss(pred['duration'], y['duration'])
            ]
        return sum([l * w for l, w in zip(losses, self.weight)])

关于loss权重的设置
可以先预先运行一下，得到几个损失值，然后手动设置权重使他们比较相近

# 训练模型

首先加载模型到GPU(如果可行)

设置损失后汉书，优化器

循环每个**epoch**，用`model.train()`将模型设置为训练模式

在每个**epoch**中，遍历`loader`来获取数据`batch`

将数据放在GPU/CPU上后，输入进模型，计算损失

将优化器的导数记录清零，再对`loss`求导，然后用`optimizer.step()`优化模型参数
<!-- 
————————————————
版权声明：本文为CSDN博主「CaptainHarryChen」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/can919/article/details/122793127 -->

In [11]:
model = MusicProducter().to(device)
loss_fn = MPLoss(loss_weight).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model)
print(loss_fn)

MusicProducter(
  (lstm): LSTM(3, 128, batch_first=True)
  (pitch_linear): Linear(in_features=128, out_features=128, bias=True)
  (pitch_sigmoid): Sigmoid()
  (step_linear): Linear(in_features=128, out_features=1, bias=True)
  (duration_linear): Linear(in_features=128, out_features=1, bias=True)
)
MPLoss(
  (pitch_loss): CrossEntropyLoss()
)


In [34]:
print("Start trainning...")
size = len(loader.dataset)
for t in range(epochs):
    model.train()
    avg_loss = 0.0
    print(f"Epoch {t+1}\n--------------------------")
    for batch,(X, y) in enumerate(tqdm(loader)):
        X = X.to(device)
        for feature in y.keys():
            y[feature] = y[feature].to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        avg_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_loss /= len(loader)
    print(f"average loss = {avg_loss}")
    if (t + 1) % 10 == 0:
        torch.save(model.state_dict(), save_model_name)
print("Done!")

Start trainning...
Epoch 1
--------------------------


100%|██████████| 105648/105648 [1:14:38<00:00, 23.59it/s]


average loss = 1.6569215441885707
Epoch 2
--------------------------


 66%|██████▌   | 69569/105648 [38:11<19:48, 30.36it/s]  


KeyboardInterrupt: 

In [35]:
torch.save(model.state_dict(), save_model_name)
print(f"Saved PyTorch Model State to {save_model_name}")

Saved PyTorch Model State to music_producter.pth


# 生成音乐

## 预测下一个音符

首先用`model.eval()`将模型设置为测试模式，用`torch.no_grad()`让`pyTorch`不记录导数节约内存

读入的音符序列需要增加一个维度来代表**batch**，因为模型的输入是带有**batch**维度的

使用`torch.tensor.unsqueeze()`来增加维度（`[0,1,2]-->[[0,1,2]]`）
然后将输入数据扔进模型里得到**predictions**

根据**prediction**中音高`pitch`的128位权重输出，按权重随机产生音符，这里我手写了一个按权值随机

由于输出中`pitch`,`duration`,`step`都是带有一维**batch**的，所以使用`np.squeeze`把**batch**维度去掉（`[[2]]–>[2]`）
最后要将`step`与`duration`与`0`取`max`，防止输出负数时间
<!-- ————————————————
版权声明：本文为CSDN博主「CaptainHarryChen」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/can919/article/details/122793127 -->

In [14]:
# def WeightedRandom(weight, k=100000) -> int:
#     sum = int(0)
#     for w in weight:
#         sum += int(k * w)
    
#     x = random.randint(1, sum)

#     sum = 0

#     for id, w in enumerate(weight):
#         sum += int(k * w)
#         if sum >= x:
#             return id
#     return x

def WeightedRandom(weight, k=100000) -> int:
    max = 0
    max_id = 0
    for id, w in enumerate(weight):
        if w > max:
            max = w
            max_id = id
    
    return max_id


# def PredictNextNote(model: MusicProducter, input: nd.ndarray):
def PredictNextNote(model: MusicProducter, input: np.ndarray, pre_hiddens):
    model.eval()
    with torch.no_grad():
        input = torch.tensor(input, dtype=torch.float32).unsqueeze(0)
        pred, hiddens = model(input, pre_hiddens)
        pitch = WeightedRandom(np.squeeze(pred['pitch'], axis=0))
        step = np.maximum(np.squeeze(pred['step'], axis=0), 0)
        duration = np.maximum(np.squeeze(pred['duration'], axis=0), 0)
    return pitch, float(step), float(duration), hiddens

## 生成序列

首先需要一个起始的输入序列作为灵感

用`sample_file_name`中初始化一个`Dataset`，然后将他的最后一个序列作为输入

具体操作就是每预测一个音符，就先删除输入序列的第一个音符，并将生成的音符放进输入序列的末尾

In [15]:
def CreateMIDIInstrumennt(notes: np.ndarray, instrument_name: str) -> pm.Instrument:
    instrument = pm.Instrument(pm.instrument_name_to_program(instrument_name))
    prev_start = 0
    for note in notes:
        prev_start += note[1]
        note = pm.Note(start=prev_start, end=prev_start +
                    note[2], pitch=note[0], velocity=100)
        instrument.notes.append(note)
    return instrument

In [16]:
sample_file_name = "sample.midi"
output_file_name = "output0.midi"
# save_model_name = "music_producter.pth"
save_model_name = "model110.pth"
predict_length = 128
sequence_length = 25

model = MusicProducter()
model.load_state_dict(torch.load(save_model_name, map_location=torch.device(device)))

sample_data = SequenceMIDI(sample_file_name, sequence_length)

Find 1 files.
Reading 1 files...


100%|██████████| 1/1 [00:00<00:00, 170.51it/s]


In [13]:
filenames = glob.glob("sample.midi")
filenames

['sample.midi']

In [43]:
sample_data

In [19]:
cur = sample_data.getendseq()
res = []
prev_start = 0

In [20]:
example = torch.tensor(cur, dtype=torch.float32).unsqueeze(0)

In [21]:
export_model = MusicProducterExport()
export_model.load_state_dict(torch.load(save_model_name, map_location=torch.device(device)))
trace_cell = torch.jit.script(export_model, example)
trace_cell.save("music_producter.pt")

/usr/lib/python3.10/site-packages/torch/jit/_script.py:1243: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


In [22]:
print(trace_cell.code)

def forward(self,
    x: Tensor,
    hidden_prev: Tensor) -> Tuple[Tuple[Tensor, Tensor, Tensor], Tuple[Tensor, Tensor]]:
  lstm = self.lstm
  x0, hidden_prev0, = (lstm).forward__0(x, None, )
  pitch_sigmoid = self.pitch_sigmoid
  pitch_linear = self.pitch_linear
  _0 = (pitch_linear).forward(torch.select(torch.slice(x0), 1, -1), )
  pitch = (pitch_sigmoid).forward(_0, )
  step_linear = self.step_linear
  step = (step_linear).forward(torch.select(torch.slice(x0), 1, -1), )
  duration_linear = self.duration_linear
  duration = (duration_linear).forward(torch.select(torch.slice(x0), 1, -1), )
  _1 = ((pitch, step, duration), hidden_prev0)
  return _1



In [42]:
torch.tensor(cur[0:3], dtype=torch.float32).unsqueeze(0)

tensor([[[0.4453, 0.0000, 0.9490],
         [0.5391, 0.5000, 0.4740],
         [0.3750, 0.5000, 0.9490]]])

In [31]:
init_hiddens = (torch.zeros(1, 1, 128), torch.zeros(1, 1, 128))

In [32]:
PredictNextNote(model, cur[0:3], init_hiddens)

(62,
 0.3219711184501648,
 0.739488959312439,
 (tensor([[[ 6.6237e-02, -1.0528e-02, -4.1737e-03,  3.0012e-02, -3.5687e-05,
            -4.4774e-03,  4.5589e-02,  9.1207e-03, -7.5694e-02, -8.3220e-03,
            -7.0098e-03, -4.2023e-03, -1.4621e-02,  3.7797e-02, -1.8497e-02,
            -1.4700e-02, -2.1675e-02, -6.3841e-02,  1.3360e-02, -5.1731e-03,
            -3.4806e-03,  9.6619e-03, -3.5108e-02,  9.9738e-02,  8.5153e-03,
             1.8953e-03, -1.3883e-01,  8.9699e-02,  7.8609e-02,  2.4018e-02,
             3.6980e-02,  1.6067e-02,  7.8808e-03, -1.5457e-03, -7.1364e-02,
            -1.5119e-04,  6.2871e-03, -4.7319e-03,  1.2844e-02,  5.9678e-02,
             2.1632e-02,  3.0234e-02, -1.6232e-02, -2.3629e-02,  3.1814e-02,
            -4.1407e-03, -2.1749e-03,  6.1773e-02, -1.2830e-02,  3.7106e-02,
            -3.6351e-03, -3.3627e-03,  6.2467e-03,  4.2484e-03, -1.1239e-03,
             5.7519e-02, -3.3000e-02,  4.2356e-03, -4.9719e-03,  2.1338e-02,
             9.7860e-03, -8.26

In [55]:
prev_hiddens = init_hiddens

for i in tqdm(range(predict_length)):
    pitch, step, duration, prev_hiddens = PredictNextNote(model, cur, pre_hiddens)
    res.append([pitch, step, duration])
    cur = cur[1:]
    cur = np.append(cur, [[pitch, step, duration]], axis=0)
    prev_start += step

pm_output = pm.PrettyMIDI()
pm_output.instruments.append(
    CreateMIDIInstrumennt(res, "Acoustic Grand Piano")
)
pm_output.write(output_file_name)

 31%|███▏      | 40/128 [00:00<00:00, 200.35it/s]

row pitch predict data: tensor([[0.0415, 0.0423, 0.0430, 0.0382, 0.0411, 0.0398, 0.0411, 0.0415, 0.0384,
         0.0408, 0.0394, 0.0413, 0.0404, 0.0414, 0.0449, 0.0420, 0.0410, 0.0413,
         0.0457, 0.0396, 0.0424, 0.0429, 0.0436, 0.0405, 0.0423, 0.0463, 0.0454,
         0.0428, 0.0486, 0.0566, 0.0615, 0.0848, 0.0893, 0.1406, 0.1225, 0.1437,
         0.4720, 0.3175, 0.4729, 0.4652, 0.5533, 0.7202, 0.6014, 0.8437, 0.8128,
         0.8675, 0.8573, 0.8584, 0.9564, 0.9158, 0.9689, 0.9607, 0.9684, 0.9746,
         0.9652, 0.9840, 0.9777, 0.9841, 0.9843, 0.9789, 0.9867, 0.9808, 0.9873,
         0.9849, 0.9853, 0.9860, 0.9787, 0.9869, 0.9803, 0.9839, 0.9793, 0.9747,
         0.9818, 0.9720, 0.9783, 0.9671, 0.9623, 0.9592, 0.9183, 0.9415, 0.8714,
         0.8500, 0.5919, 0.4195, 0.4499, 0.2540, 0.2604, 0.1731, 0.1674, 0.1405,
         0.1081, 0.1052, 0.0815, 0.0707, 0.0639, 0.0552, 0.0480, 0.0472, 0.0469,
         0.0458, 0.0409, 0.0416, 0.0409, 0.0417, 0.0410, 0.0411, 0.0411, 0.0419,
    

 67%|██████▋   | 86/128 [00:00<00:00, 219.37it/s]

row pitch predict data: tensor([[0.0268, 0.0320, 0.0308, 0.0406, 0.0314, 0.0223, 0.0324, 0.0288, 0.0371,
         0.0263, 0.0269, 0.0285, 0.0184, 0.0219, 0.0194, 0.0211, 0.0324, 0.0373,
         0.0225, 0.0260, 0.0261, 0.0236, 0.0287, 0.0238, 0.0315, 0.0302, 0.0332,
         0.0324, 0.0169, 0.0166, 0.0347, 0.0287, 0.0282, 0.0432, 0.0419, 0.0462,
         0.0231, 0.0367, 0.0254, 0.0294, 0.0349, 0.0249, 0.0452, 0.1087, 0.0832,
         0.2604, 0.1997, 0.1764, 0.9221, 0.5845, 0.9579, 0.9143, 0.9576, 0.9805,
         0.9684, 0.9784, 0.9674, 0.9827, 0.9761, 0.9824, 0.9889, 0.9827, 0.9902,
         0.9845, 0.9874, 0.9875, 0.9872, 0.9766, 0.9879, 0.9910, 0.9862, 0.9898,
         0.9914, 0.9826, 0.9901, 0.9874, 0.9891, 0.9851, 0.9813, 0.9877, 0.9872,
         0.9860, 0.9859, 0.9906, 0.9773, 0.9890, 0.9931, 0.9912, 0.9740, 0.9497,
         0.6150, 0.5023, 0.3087, 0.2079, 0.0838, 0.0635, 0.0402, 0.0801, 0.0330,
         0.0394, 0.0323, 0.0435, 0.0255, 0.0320, 0.0379, 0.0260, 0.0356, 0.0230,
    

100%|██████████| 128/128 [00:00<00:00, 219.62it/s]

row pitch predict data: tensor([[0.0274, 0.0324, 0.0313, 0.0410, 0.0318, 0.0227, 0.0328, 0.0296, 0.0380,
         0.0270, 0.0276, 0.0292, 0.0190, 0.0226, 0.0199, 0.0219, 0.0335, 0.0380,
         0.0227, 0.0262, 0.0269, 0.0245, 0.0296, 0.0241, 0.0320, 0.0313, 0.0336,
         0.0331, 0.0176, 0.0167, 0.0358, 0.0297, 0.0293, 0.0450, 0.0428, 0.0480,
         0.0237, 0.0375, 0.0264, 0.0300, 0.0361, 0.0258, 0.0472, 0.1097, 0.0843,
         0.2620, 0.2031, 0.1801, 0.9222, 0.5833, 0.9562, 0.9129, 0.9564, 0.9804,
         0.9677, 0.9781, 0.9670, 0.9823, 0.9757, 0.9815, 0.9885, 0.9818, 0.9900,
         0.9840, 0.9870, 0.9872, 0.9869, 0.9755, 0.9873, 0.9906, 0.9856, 0.9893,
         0.9912, 0.9821, 0.9897, 0.9870, 0.9886, 0.9848, 0.9809, 0.9875, 0.9871,
         0.9857, 0.9856, 0.9901, 0.9761, 0.9884, 0.9929, 0.9908, 0.9731, 0.9482,
         0.6152, 0.5038, 0.3065, 0.2095, 0.0848, 0.0658, 0.0412, 0.0805, 0.0332,
         0.0406, 0.0329, 0.0441, 0.0264, 0.0327, 0.0384, 0.0270, 0.0368, 0.0236,
    